In [1]:
# df_players
# name, elo, state (H/m), move time avg (not possible yet), acpl game avg
# df_games
# p1_ID, p2_ID, winner_ID, date, opening, p1_ELO, p2_ELO
# df_moves
# player_ID, game_ID, time, eval, fen/move, comment

In [2]:
import pandas as pd
import numpy as np
import uuid #for id generation

import chess.pgn
from stockfish import Stockfish

In [3]:
# stockfish = Stockfish(
#     'stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
#     parameters={"Threads": 2, 'Min Split Depth': 26, 'Ponder':True}
# )
# stockfish.set_elo_rating(2600)
# stockfish.set_skill_level(30)

In [4]:
%%time

players = {
    'White':[],
    'White_Elo': [],
    'Black': [],
    'Black_Elo': [],
    'WhiteIsComp':[],
}

games = {
    'Game_ID': [],
    'Date' : [],
    'White':[],  # Dummy ID
    'White_Elo': [],
    'Black': [],  # Dummy ID
    'Black_Elo': [],
    'ECO': [],
    'Result': [],
}

moves_log_dict = {
    'Game_ID': [],
    'White': [],  # Dummy ID
    'Black': [],  # Dummy ID
    'ECO': [],
    'FEN_moves': [],
    #'cpl': [],
    'WhiteIsComp': [],
    'Result': [],
}

# read file
pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player
game_counter = 0

while True:  # keep reading games
    try:
        game = chess.pgn.read_game(pgn)
        board = game.board()
        moves = list(game.mainline_moves())
        
        # Player
        players['White_Elo'].append(game.headers['WhiteElo'])
        players['Black_Elo'].append(game.headers['BlackElo'])
        players['White'].append(game.headers['White'])
        players['Black'].append(game.headers['Black'])
        players['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        
        # Games
        games['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
        games['White'].append(game.headers['White'])  # dummy ID
        games['Black'].append(game.headers['Black'])  # dummy ID
        games['White_Elo'].append(game.headers['WhiteElo'])
        games['Black_Elo'].append(game.headers['BlackElo'])
        games['ECO'].append(game.headers['ECO'])
        games['Result'].append(game.headers['Result'])
        games['Date'].append(game.headers['Date'])
        
        # Moves
        fen_pos = []
        acpl = []
        
        moves_log_dict['Game_ID'].append(game.headers['FICSGamesDBGameNo'])  # dummy ID for this dataset only breaks
        moves_log_dict['White'].append(game.headers['White'])  # dummy ID
        moves_log_dict['Black'].append(game.headers['Black'])  # dummy ID
        moves_log_dict['ECO'].append(game.headers['ECO'])
        moves_log_dict['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        moves_log_dict['Result'].append(game.headers['Result'])

        # MOVE CYCLE
        for move in moves:
            board.push(move)
            fen_pos.append(board.fen())
            #stockfish.set_fen_position(board.fen())  # load stockfish with current FEN for eval
#             cpl = stockfish.get_evaluation()['value']/100
#             acpl.append(cpl)
        
        moves_log_dict['FEN_moves'].append(fen_pos)
        #moves_log_dict['cpl'].append(acpl)
        
        game_counter += 1
        if game_counter == 50:  # number of games to read
            break
    except AttributeError:  # no further games to read
        print('No further games to load.')
        break

print(f'{game_counter} games read.')
#TODO takes ~1 sec to process 5 games, too slow.

50 games read.
CPU times: user 424 ms, sys: 0 ns, total: 424 ms
Wall time: 423 ms


In [5]:
#generates unique IDs
def id_generator(id):
    return uuid.uuid4().int

## df_players

In [6]:
df_players = pd.DataFrame(players)
df_players.head()

,White,White_Elo,Black,Black_Elo,WhiteIsComp
0,forlat,1970,Geforce,2204,Yes
1,Geforce,2201,forlat,1973,No
2,forlat,1976,Geforce,2198,Yes
3,Geforce,2211,forlat,1963,No
4,forlat,1958,Geforce,2216,Yes


In [7]:
#pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8') 

In [8]:
#game = chess.pgn.read_game(pgn)

In [9]:
Black = df_players["Black"]
White = df_players["White"]

In [10]:
#l = set.union

In [11]:
l = list(Black) + list(White)
players_names = list(set(l))

In [12]:
df_pl = {'Players': players_names}
players_unique = pd.DataFrame(df_pl)
players_unique['Player_ID'] = players_unique['Players'].apply(id_generator)

In [13]:
players_unique

,Players,Player_ID
0,Ghannoum,112808929070435947929555899724917396256
1,RamMiguel,245691260732732474796050907211541068018
2,nakshatra,85622381157584195498058867492324741020
3,IFDThor,270612889416003223152324969538463191190
4,Qiyas,141112977831341997884817163745753577015
5,Geforce,167326815588201837430298469719180834781
6,Arsyah,264378418421267008838391603410731157629
7,foggydew,291283725545504362506374092266672026852
8,JMM,221493385404289799630922669471676884438
9,scalaQueen,201373412425016114495222131263353875705


In [14]:
m_white = df_players.merge(players_unique, left_on=["White"], right_on=['Players'])
m_white['White_ID'] = m_white['Player_ID']
m_white.drop(columns=['Players', "Player_ID"], inplace=True)

In [23]:
m_white

,White,White_Elo,Black,Black_Elo,WhiteIsComp,White_ID
0,forlat,1970,Geforce,2204,Yes,321467545545008302936834840814226732329
1,forlat,1976,Geforce,2198,Yes,321467545545008302936834840814226732329
2,forlat,1958,Geforce,2216,Yes,321467545545008302936834840814226732329
3,forlat,1964,Geforce,2210,Yes,321467545545008302936834840814226732329
4,forlat,1970,Geforce,2204,Yes,321467545545008302936834840814226732329
5,forlat,1976,Geforce,2198,Yes,321467545545008302936834840814226732329
6,forlat,1976,Geforce,2194,Yes,321467545545008302936834840814226732329
7,forlat,1984,Geforce,2186,Yes,321467545545008302936834840814226732329
8,forlat,1976,Geforce,2186,Yes,321467545545008302936834840814226732329
9,forlat,1984,Geforce,2178,Yes,321467545545008302936834840814226732329


In [30]:
m_black = m_white.merge(players_unique, left_on=["Black"], right_on=['Players'])
m_black['Black_ID'] = m_black['Player_ID']
m_black.drop(columns=['Players', "Player_ID"], inplace=True)

In [31]:
df_players = m_black
df_players

,White,White_Elo,Black,Black_Elo,WhiteIsComp,White_ID,Black_ID
0,forlat,1970,Geforce,2204,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
1,forlat,1976,Geforce,2198,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
2,forlat,1958,Geforce,2216,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
3,forlat,1964,Geforce,2210,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
4,forlat,1970,Geforce,2204,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
5,forlat,1976,Geforce,2198,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
6,forlat,1976,Geforce,2194,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
7,forlat,1984,Geforce,2186,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
8,forlat,1976,Geforce,2186,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
9,forlat,1984,Geforce,2178,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781


## df_games

In [19]:
df_games = pd.DataFrame(games)
df_games['Game_ID'] = df_games['Game_ID'].apply(id_generator)
##TODO date to datetime
df_games

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result
0,27802410499662145447418801284695169290,2021.01.31,forlat,1970,Geforce,2204,A01,0-1
1,273677539331814060354795956337282283867,2021.01.31,Geforce,2201,forlat,1973,B00,1-0
2,128629789545662653076084930132381091210,2021.01.31,forlat,1976,Geforce,2198,A20,0-1
3,153340052511985101896965336996969936917,2021.01.31,Geforce,2211,forlat,1963,B10,0-1
4,269997004028306991168990729844947043215,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2
5,232445541182478126269233476011025611494,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0
6,152781445678667442610001456543883333440,2021.01.31,Geforce,2213,forlat,1961,B00,1-0
7,234199072182218512385604485238043334391,2021.01.31,forlat,1964,Geforce,2210,C28,0-1
8,169460409157164818758495890178441586966,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1
9,113545879279653919805345011711011341145,2021.01.31,Geforce,2207,forlat,1967,B10,1-0


## df_moves

In [20]:
df_moves = pd.DataFrame(moves_log_dict)
#TODO moves and cpl as features when loading to DB
df_moves['Game_ID'] = df_games['Game_ID']
df_moves.head(20)

,Game_ID,White,Black,ECO,FEN_moves,WhiteIsComp,Result
0,27802410499662145447418801284695169290,forlat,Geforce,A01,[rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR...,Yes,0-1
1,273677539331814060354795956337282283867,Geforce,forlat,B00,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0
2,128629789545662653076084930132381091210,forlat,Geforce,A20,[rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR...,Yes,0-1
3,153340052511985101896965336996969936917,Geforce,forlat,B10,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,0-1
4,269997004028306991168990729844947043215,forlat,Geforce,D30,[rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR...,Yes,1/2-1/2
5,232445541182478126269233476011025611494,exeComp,Ruvarashe,B92,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,Yes,1-0
6,152781445678667442610001456543883333440,Geforce,forlat,B00,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0
7,234199072182218512385604485238043334391,forlat,Geforce,C28,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,Yes,0-1
8,169460409157164818758495890178441586966,Ruvarashe,exeComp,C11,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,0-1
9,113545879279653919805345011711011341145,Geforce,forlat,B10,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0


# search

In [32]:
def search_df(df, column, value):
    try:
        index = np.where(df[column] == value)
        df = df.iloc[index]
        if len(df) == 0:
            return f'No games found for {value}.'
        return df
    except KeyError as e:
        print(e, f'not found.')

In [34]:
search_df(df_players, 'White', 'forlat')

,White,White_Elo,Black,Black_Elo,WhiteIsComp,White_ID,Black_ID
0,forlat,1970,Geforce,2204,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
1,forlat,1976,Geforce,2198,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
2,forlat,1958,Geforce,2216,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
3,forlat,1964,Geforce,2210,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
4,forlat,1970,Geforce,2204,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
5,forlat,1976,Geforce,2198,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
6,forlat,1976,Geforce,2194,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
7,forlat,1984,Geforce,2186,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
8,forlat,1976,Geforce,2186,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781
9,forlat,1984,Geforce,2178,Yes,321467545545008302936834840814226732329,167326815588201837430298469719180834781


In [ ]:
def transform_new_df(input_df, players_unique_df):
    input_df
    # 1. update players unique df
        # extract new player names
        # create df with new players only and assign ids
        # concat players unique df and new players df -> updated_players_unique_df
    
    # 2. merge input df with players ids
        # 2.1 add a column for white players
        # 2.2 add a column for black players
    return transformed_input_df, updated_players_unique_df
    
        
        
        
        